In [ ]:
with open('libraries.py') as f:
    code = f.read()
exec(code)

with open('functions.py') as f:
    code = f.read()
exec(code)

In [ ]:
# determine user
user = getpass.getuser()
if user == 'peymanshahidi':
    main_folder_path = '/Users/peymanshahidi/Dropbox (MIT)/Research/AI and Occupations/ai-exposure'
    data_path = f'{main_folder_path}/output/sparseVsDense_DAGs'

In [ ]:
DAG_dict = {'Manual DAG': 'M',
           'Naive DAG': 'N',
           'Conditioned Naive DAG': 'CN',
           'First-Last Task DAG': 'FLT',
           'Conditioned First-Last Task DAG': 'CFLT',
           'Partitioned DAG': 'P',
           'Conditioned Partitioned DAG': 'CP'}

In [ ]:
data_v1 = pd.read_csv(f'{data_path}/costMin_stats_1.csv')
data_v2 = pd.read_csv(f'{data_path}/costMin_stats.csv')
data_v2 = data_v2.dropna(how='all')

data = pd.concat([data_v1, data_v2], axis=0)
data['exec_time_sec'] = data['exec_time_min'] * 60
data['DAG_type'] = data['DAG_type'].replace(DAG_dict)
data#.iloc[290:300]

In [ ]:
import matplotlib.cm as cm

output_col = 'exec_time_sec'
x_axis_col = 'DAG_sparsity'

# Get unique values of alpha and occupations
#data = data.iloc[-100:]
unique_alphas = sorted(data['alpha'].unique())
unique_occupations = data['occupation'].unique()

# Create subplots
fig, axes = plt.subplots(nrows=len(unique_occupations), ncols=len(unique_alphas), figsize=(18, 13 * len(unique_alphas)))

# Loop through each unique alpha and occupation to create subplots
for i, occupation in enumerate(unique_occupations):
    for j, alpha in enumerate(unique_alphas):
        ax = axes[i, j] if len(unique_alphas) > 1 else axes[j]  # Adjust indexing for single alpha case
        subset_alpha = data[data['alpha'] == alpha]
        subset_occupation = subset_alpha[subset_alpha['occupation'] == occupation]
        subset_occupation[f'{output_col}'] = subset_occupation[f'{output_col}'] / subset_occupation[f'{output_col}'].min()
        unique_DAG_types = subset_occupation['DAG_type'].unique()
        n = int(subset_occupation['num_tasks'].iloc[0])

        # Create a colormap for the DAG types
        colors = cm.get_cmap('tab10', len(unique_DAG_types))

        # Plot each DAG type within the occupation
        for k, DAG_type in enumerate(unique_DAG_types):
            subset_DAG_type = subset_occupation[subset_occupation['DAG_type'] == DAG_type]
            ax.scatter(subset_DAG_type[f'{x_axis_col}'], subset_DAG_type[f'{output_col}'], label=DAG_type, color=colors(k))

        # Set labels and title for each subplot
        ax.set_title(f'{occupation} ({n})\nalpha: {alpha}')
        ax.legend(loc='best')
        ax.set_xlabel(f'{x_axis_col}')
        if j == 0:
            ax.set_ylabel(f'{output_col}')
        

# Adjust layout
plt.tight_layout()
plt.savefig(f'{data_path}/scatterplot_{output_col}_vs_{x_axis_col}.png', dpi=100)
plt.show()

In [ ]:
# Calculate the average of "exec_time_sec" over "DAG_type" for each alpha
average_df = df.groupby(['alpha', 'DAG_type'])['exec_time_sec'].mean().reset_index()

# Get unique alpha values
unique_alphas = sorted(average_df['alpha'].unique())

# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=4, figsize=(20, 5))  # 4 subplots in a single row

# Loop through each unique alpha and create a scatter plot in each subplot
for i, alpha in enumerate(unique_alphas):
    ax = axes[i]  # Select the subplot axis
    subset = average_df[average_df['alpha'] == alpha]
    ax.scatter(subset['DAG_type'], subset['exec_time_sec'], s=30)  # s=100 sets marker size
    ax.set_title(f'Alpha = {alpha}')
    ax.set_xlabel('DAG Type')
    ax.set_ylabel('Average Exec Time (sec)')
    ax.grid(visible=True, linestyle='--', alpha=0.5)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()

In [ ]:
# Calculate the average of "exec_time_sec" over "alpha" for each DAG_type
average_df = df.groupby(['DAG_type', 'alpha'])['exec_time_sec'].mean().reset_index()

# Get unique DAG_type values
unique_DAG_types = average_df['DAG_type'].unique()

# Create subplots
fig, axes = plt.subplots(nrows=1, ncols=len(unique_DAG_types), figsize=(20, 5))  # 4 subplots in a single row

# Loop through each unique DAG_type and create a scatter plot in each subplot
for i, dag_type in enumerate(unique_DAG_types):
    ax = axes[i]  # Select the subplot axis
    subset = average_df[average_df['DAG_type'] == dag_type]
    ax.scatter(subset['alpha'], subset['exec_time_sec'], s=30)  # s=100 sets marker size
    ax.set_xscale('log')  # Set x-axis to log scale
    ax.set_title(f'DAG Type = {dag_type}')
    ax.set_xlabel('Alpha')
    ax.set_ylabel('Average Exec Time (sec)')
    ax.grid(visible=True, linestyle='--', alpha=0.5)

# Adjust layout to prevent overlap
plt.tight_layout()
plt.show()